## 1. Make a simple web server

In [ ]:
const handler = (req) => {
  // Create a new response object
  const body = new TextEncoder().encode("Hello World!");
  return new Response(body, { status: 200 });
};

In [ ]:
let server = Deno.serve({ port: 8001 }, handler);

In [ ]:
const response = await fetch("http://localhost:8001")

In [ ]:
await response.text()

In [ ]:
await server.shutdown()

### Set up the documents, index and query engine

In [ ]:
import * as mod from "https://deno.land/std@0.213.0/dotenv/mod.ts";
import { 
    Document, 
    VectorStoreIndex, 
    SimpleDirectoryReader 
} from "npm:llamaindex@0.1.8"
const keys = await mod.load({export:true}) // read API key from .env

const documents = await new SimpleDirectoryReader().loadData({directoryPath: "./data"})
const index = await VectorStoreIndex.fromDocuments(documents)
const queryEngine = index.asQueryEngine()

### Create a new handler

This new handler will accept a query as input and respond from the query engine.

In [ ]:
const handler2 = async (req) => {
    if(req.method == "POST") {
        // we'll expect the incoming query to be a JSON object of the form {query: ...}
        let data = await req.json()
        let answer = await queryEngine.query({query: data.query})
        // and our response will be a JSON object of the form {response: ...}
        let responseObj = {
            response: answer.toString()
        }
        return new Response(JSON.stringify(responseObj), { 
            status: 200
        })
    } else {
        return new Response("Not found", { status: 404 })
    }
}
let server2 = Deno.serve( { port: 8002 }, handler2 )

### Send a request to the server

In [ ]:
let data = { query: "How does the author feel about college?" }

In [ ]:
let response2 = await fetch("http://localhost:8002", {
  method: "POST",
  headers: {
    "Content-Type": "application/json"
  },
  body: JSON.stringify(data) // Convert the JavaScript object to a JSON string
})

In [ ]:
let responseObj = await response2.json()
console.log(responseObj.response)

### Shut down the server

In [ ]:
await server2.shutdown()

## 2. Make a web app!

In this next section, we're going to use a development version of a web app, which is already setup for you.

- Use the utility functions in **utils.ts** file. 

In [ ]:
import { runFrontend, runBackend, addToFrontend } from './utils.ts'

In [ ]:
runFrontend()

### Runa API server

In [ ]:
runBackend()

In [ ]:
// import-me: 1
import React, { useState } from 'https://esm.sh/react@17.0.2';

const Counter: React.FC = () => {
  return (
    <div>
      <h1>Counter: 0</h1>
      <button>Increment</button>
    </div>
  );
};

export default Counter;

In [ ]:
await addToFrontend("1")

In [ ]:
// import-me: 2
import React, { useState } from 'https://esm.sh/react@17.0.2';

const Counter: React.FC = () => {
  const count = 10

  return (
    <div>
      <h1>Counter: {count}</h1>
      <button>Increment</button>
    </div>
  );
};

export default Counter;

In [ ]:
await addToFrontend("2")

In [ ]:
// import-me: 3
import React, { useState } from 'https://esm.sh/react@17.0.2';

const Counter: React.FC = () => {
  const [count, setCount] = useState<number>(0);

  const increment = () => {
    let newCount = count+1
    setCount(newCount);
  };

  return (
    <div>
      <h1>Counter: {count}</h1>
      <button onClick={increment}>Increment</button>
    </div>
  );
};

export default Counter;

In [ ]:
await addToFrontend("3")

In [ ]:
// import-me: 4
import React, { useState } from 'https://esm.sh/react@17.0.2';

const QuerySender: React.FC = () => {
  return (
    <div>
      <h1>Ask a question</h1>
      <form>
        <input id="query" type="text"/>
        <button type="submit">Query</button>
      </form>
      <div id="answer">Answer will go here</div>
    </div>
  );
};

export default QuerySender;

In [ ]:
await addToFrontend("4")

In [ ]:
// import-me: 5
import React, { useState } from 'https://esm.sh/react@17.0.2';

const QuerySender: React.FC = () => {
  const [query, setQuery] = useState<string>('');    

  // Function to update the state with the input value
  const handleChange = (e) => {
    setQuery(e.target.value);
  };
    
  return (
    <div>
      <h1>Ask a question</h1>
      <form>
        <input id="query" type="text" value={query} onChange={handleChange}/>
        <button type="submit">Query</button>
      </form>
      <div id="answer">{query}</div>
    </div>
  );
};

export default QuerySender;

In [ ]:
await addToFrontend("5")

In [ ]:
// import-me: 6
import React, { useState } from 'https://esm.sh/react@17.0.2';

const QuerySender: React.FC = () => {
  const [query, setQuery] = useState<string>('');    
  const [answer, setAnswer] = useState<string>('');

  const handleSubmit = async (e) => {
    e.preventDefault()
    setAnswer(query + " is the answer");
  }
    
  // Function to update the state with the input value
  const handleChange = (e) => {
    setQuery(e.target.value);
  };
    
  return (
    <div>
      <h1>Ask a question</h1>
      <form onSubmit={handleSubmit}>
        <input id="query" type="text" value={query} onChange={handleChange}/>
        <button type="submit">Query</button>
      </form>
      <div id="answer">{answer}</div>
    </div>
  );
};

export default QuerySender;

In [ ]:
await addToFrontend("6")

In [ ]:
// import-me: 7
import React, { useState } from 'https://esm.sh/react@17.0.2';

const QuerySender: React.FC = () => {
  const [query, setQuery] = useState<string>('');
  const [answer, setAnswer] = useState<string>('');

  const handleSubmit = async (e) => {
    e.preventDefault()
    setAnswer("Thinking...")
    const response = await fetch('http://localhost:8000', {
      method: 'POST',
      headers: {
        'Content-Type': 'application/json',
      },
      body: JSON.stringify({ query }),
    });
    const data = await response.json();
    console.log('Response from the server:', data);
    setAnswer(data.response);
  }

  // Function to update the state with the input value
  const handleChange = (e) => {
    setQuery(e.target.value);
  };  

  return (
    <div>
      <h1>Ask a question</h1>
      <form onSubmit={handleSubmit}>
        <input id="query" type="text" value={query} onChange={handleChange} />
        <button type="submit">Query</button>
      </form>
      <div id="answer">{answer}</div>
    </div>
  );
};

export default QuerySender;


In [ ]:
await addToFrontend("7")